<a href="https://colab.research.google.com/github/YS-JEOUNG/amazon-review-project/blob/main/code/artscrafts_code/artscrafts_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import gzip
import dask as dd

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# 이전 과정은 이미 맨 처음에 했던 것이므로 패스. 바로 해당 파일 저장본 불러오겠음.
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/arts_total.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130133 entries, 0 to 130132
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   overall           130133 non-null  float64
 1   vote              130133 non-null  int64  
 2   verified          130133 non-null  bool   
 3   reviewer_id       130133 non-null  object 
 4   asin              130133 non-null  object 
 5   style             130133 non-null  object 
 6   reviewer_name     130128 non-null  object 
 7   unix_review_time  130133 non-null  int64  
 8   category          130133 non-null  object 
 9   description       130133 non-null  object 
 10  title             130133 non-null  object 
 11  brand             129770 non-null  object 
 12  rank              130133 non-null  object 
 13  main_cat          129791 non-null  object 
 14  date              20908 non-null   object 
 15  price             115105 non-null  object 
 16  review_text       13

In [ ]:
df = df.drop(['vote', 'verified', 'style', 'reviewer_name', 'unix_review_time', 'category',
              'description', 'brand', 'rank', 'main_cat', 'date', 'price', 'review_text',
              'rating_class', 'time'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130133 entries, 0 to 130132
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   overall      130133 non-null  float64
 1   reviewer_id  130133 non-null  object 
 2   asin         130133 non-null  object 
 3   title        130133 non-null  object 
dtypes: float64(1), object(3)
memory usage: 4.0+ MB


In [ ]:
counts_reviewer=df['reviewer_id'].value_counts()
counts_product=df['asin'].value_counts()

In [ ]:
counts_reviewer

A1PAGHECG401K1    55
ALYZJ7W14YS26     54
A13OFOB1394G31    47
A2GJX2KCUSR0EI    45
A1GYEGLX3P2Y7P    37
                  ..
A2HVJLHQFK152E     1
A1CT1L7C4Y1C3C     1
AN6L4MHFEU98Z      1
A3TLJWAV1WNI7V     1
A3FEDCN23EW1KF     1
Name: reviewer_id, Length: 111107, dtype: int64

In [ ]:
counts_product

B005HR1JMO    728
B004RDH7Y8    710
B003AVMZA4    708
B000F7DPEQ    575
B01D3MWPU4    420
             ... 
B00433201C      1
B00WUC3B6G      1
B013FHO9X8      1
B01E391UVG      1
B000GERNNM      1
Name: asin, Length: 14060, dtype: int64

In [ ]:
len(counts_reviewer[counts_reviewer>=50].index)

2

In [ ]:
len(counts_product[counts_product>=50].index)

398

In [ ]:
# 얘는 왜 한걸까?
df1 = df[df['reviewer_id'].isin(counts_reviewer[counts_reviewer>=50].index)]
df1.shape

(109, 4)

In [ ]:
df1 = df[df['asin'].isin(counts_product[counts_product>=50].index)]
df1.shape

(38597, 4)

In [ ]:
df1.head(3)

,overall,reviewer_id,asin,title
96,5.0,A33QFZUZ8YHYX,B0000302YN,"Dremel 290-01 0.2 Amp 7,200 Stroke Per Minute ..."
97,2.0,A1H7LBUH5BSJRV,B0000302YN,"Dremel 290-01 0.2 Amp 7,200 Stroke Per Minute ..."
98,1.0,A6SVJNBB9UZLF,B0000302YN,"Dremel 290-01 0.2 Amp 7,200 Stroke Per Minute ..."


In [ ]:
ratings_sum = pd.DataFrame(df1.groupby(['asin'])['overall'].sum()).rename(columns = {'overall': 'ratings_sum'})
top10 = ratings_sum.sort_values('ratings_sum', ascending=False).head(10)
top10

,ratings_sum
asin,
B003AVMZA4,2956.0
B004RDH7Y8,2671.0
B005HR1JMO,2598.0
B000F7DPEQ,2283.0
B01D3MWPU4,1483.0
B00WW8KIRI,1411.0
B009CCVS4E,1382.0
B005GXPOAM,1338.0
B000980L02,1234.0


In [ ]:
top10_popular = top10.merge(df1, left_index=True, right_on='asin').drop_duplicates(
    ['asin', 'title'])[['asin', 'title', 'ratings_sum']]

In [ ]:
top10_popular

,asin,title,ratings_sum
41399,B003AVMZA4,Brother SE400 Combination Computerized Sewing ...,2956.0
49352,B004RDH7Y8,SINGER Quantum Stylist 9960 Computerized Porta...,2671.0
55369,B005HR1JMO,Singer | 7258 100-Stitch Computerized Sewing M...,2598.0
7378,B000F7DPEQ,Brother XL2600I Sew Advance Sew Affordable 25-...,2283.0
111128,B01D3MWPU4,Singer | Heavy Duty 4411 Sewing Machine with 1...,1483.0
95015,B00WW8KIRI,Sugru Moldable Glue - Original Formula - New C...,1411.0
64372,B009CCVS4E,PowerPress HPM-1515-BK Industrial-Quality Digi...,1382.0
54838,B005GXPOAM,Brother XR9500PRW Project Runway Limited Editi...,1338.0
3665,B000980L02,Clover Soft Jumbo Stitch Ring Markers,1234.0
38901,B002HR4CW6,Clover 3142 Yarn Threader,1231.0


In [ ]:
df1.head(3)

,overall,reviewer_id,asin,title
96,5.0,A33QFZUZ8YHYX,B0000302YN,"Dremel 290-01 0.2 Amp 7,200 Stroke Per Minute ..."
97,2.0,A1H7LBUH5BSJRV,B0000302YN,"Dremel 290-01 0.2 Amp 7,200 Stroke Per Minute ..."
98,1.0,A6SVJNBB9UZLF,B0000302YN,"Dremel 290-01 0.2 Amp 7,200 Stroke Per Minute ..."


In [ ]:
# df1.pivot() 하니까 index contains duplicate entries, cannot reshape 에러 발생
# pd.pivot_table() 이용
ratingsd = pd.pivot_table(data=df1, index='asin', columns='reviewer_id', values='overall').fillna(0)
ratingsd.head()

reviewer_id,A0193442L7IJXD7HANDM,A02049035W9C30B2NBJM,A0228801ER3IBSLNOT5W,A0345137PWD0TJ09QK1A,A0429690KG9J3JO6LSSH,A04454666ZIS4FUX7ZME,A0479295RXACIYIXRLRF,A0483767C53I9DYB5TMS,A0490453AYGB1BAAOG64,A0516005NLXNKCMT2YI9,A05848286122ZISVU9YI,A06254038DV28169YE30,A0802864SYTA93AK0XMU,A0944456Z3LN62I2DT3O,A1002BMQTTW5F8,A100434D2BHASB,A1004HHMSDY5IP,A100DKGJDUFS2I,A100FCW9HQ1I0X,A100GZTO7FXSS,A100IBOIJVW7H0,A100TXY7M5TVDW,A100UD67AHFODS,A101AAU7AWNKOS,A101DANINH0Q5M,A101GN97A47S1P,A101HP0PZ5CDQ9,A101I1C4TRSKNO,A101UPLI45LNM4,A101UVGO0HK7UQ,A101YLHRFE7879,A1029QPGAKN80T,A102D6PSQ6WBSA,A102W91CO98O9K,A102Z2PGSWNASB,A1034HJ8RNL6IT,A103IHKE6M8G6T,A1040HFRWSVPUC,A1043NRH5KAJZ3,A1047B34OFNS4H,...,AZTSEQPENAZGP,AZTT49875VXW4,AZTZ7SIIRXLXE,AZUB5DZWOP9GT,AZUBFRR5L0634,AZUEE0CPOMXIH,AZUNCDOLPFSL6,AZV2U6GU5QA6C,AZVNM2QDUFOG5,AZVPZ2GHRH6X0,AZVRA7LJO8EHB,AZW0FW3EQEJX1,AZWAE29GBXK0X,AZWD2KSANOM4K,AZWDTS5SV9GXM,AZWIWTDWY5YMA,AZWKT8OC3HAG,AZWMC45FXG08Q,AZWNYVPZYPTOZ,AZWPGMYE3MRT1,AZWR3B15A5KL,AZWX64GPQSRHR,AZWZAV7FVO3G0,AZX5A3VEXIZWU,AZXAUT22JQQDH,AZXAVVT71RR7J,AZXDJYGX4S0IV,AZXKAH2DE6C8A,AZXO0J5CHBJ1I,AZYBGMYKQH8HR,AZYKTPOHMZX7N,AZYML1QLCY4U8,AZYP4FQ2L2C4O,AZYQ60ZR354HC,AZYW27SS60XQY,AZZ1CC0BCXZ9H,AZZ74XTNQCOS4,AZZXNFW30OVPU,AZZXW4PH4OEEK,AZZZPAB0MBZ69
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
B0000302YN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004THXH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004Z2UB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004Z2UD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00006B8FS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratingsd.shape

(398, 34594)

In [ ]:
traind, testd = train_test_split(ratingsd, test_size=0.3, random_state=42)

In [ ]:
# DataFrame.as_matrix()는 지원 안 함. DataFrame.values로 해야 함.
train = traind.values
test = testd.values

In [ ]:
train.shape

(278, 34594)

In [ ]:
train_short = train[:50,:]

In [ ]:
train_short.shape

(50, 34594)

In [ ]:
sparsity = float(len(train_short.nonzero()[0]))
sparsity /= (train_short.shape[0]*train_short.shape[1])
sparsity *= 100
print(f'sparsity: {sparsity}%')

sparsity: 0.2477886338671446%


In [ ]:
def item_similarity(ratings, epsilon=1e-3):
    sim = ratings.T.dot(ratings)+epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim/norms/norms.T)

In [ ]:
# 코랩이 죽어버렸네
item_sim = item_similarity(train_short)

In [ ]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [ ]:
item_predic